In [1]:
import sys
import torch
import wandb
sys.path.append('..')
import pytorch_lightning as pl
from utils.data_modules.super_resolution import EEGSuperResolutionDataModule
from models.trainers.super_resolution import SuperResolutionTrainerModel, SuperResolutionPlottingCallback

s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


S:\PolySecLabProjects\eeg-image-decoding\data\all-joined-1\eeg\epochs


In [2]:
dm = EEGSuperResolutionDataModule(
    input_channels=['Fp1', 'Fp2', 'AF3', 'AF4', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2'],
    output_channels=['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'],
    sfreq=250,
    montage='standard_1020',
    window_before_event_ms=50,
    window_after_event_ms=600,
    subject=1, 
    session=1, 
    batch_size=64, 
    num_workers=4
)

In [3]:
sample_data = dm.get_sample_info()
epochs = 100
subject = 1
session = 1
input_channels = sample_data['input']['channel_names']
output_channels = sample_data['output']['channel_names']
timesteps = sample_data['input']['num_timesteps']

Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320
Original dataframe size: 640


In [4]:
checkpoint_path = "S:\\PolySecLabProjects\\eeg-image-decoding\\code\\models\\check_points\\super_resolution\\subj1_session1_epoch=99.ckpt"

In [5]:
lightning_model = SuperResolutionTrainerModel.load_from_checkpoint(checkpoint_path)

results_callback = SuperResolutionPlottingCallback()

logger = pl.loggers.WandbLogger(project="eeg_super_resolution")

trainer = pl.Trainer(
    max_epochs=epochs,
    callbacks=[results_callback],
    logger=logger,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1
)

trainer.test(lightning_model, dm)

s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\pytorch_lightning\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.5.2, which is newer than your current Lightning version: v2.5.1.post0
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: dubs2310 (dubs2310-cal-poly-pomona) to https://api.wandb.ai. Use `wandb login --rel

Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Original dataframe size: 640
Testing DataLoader 0: 100%|██████████| 10/10 [00:53<00:00,  0.19it/s]Callback collected test metrics: {'test_loss': 17.112995147705078, 'test_snr': 2.7484703063964844, 'test_mae': 0.35854288935661316, 'test_mse': 0.2468574345111847, 'test_nmse': 0.5162146935463189, 'test_pearson': 0.773216088149331}
Plotting test metrics: {'test_loss': 17.112995147705078, 'test_snr': 2.7484703063964844, 'test_mae': 0.35854288935661316, 'test_mse': 0.2468574345111847, 'test_nmse': 0.5162146935463189, 'test_pearson': 0.773216088149331}
Logged test metrics bar plot to wandb

=== NMSE ANALYSIS ===
NMSE (variance): 0.516215
NMSE (mean square): 0.516127
Average MSE: 0.246857
Average Target Variance: 0.478207
Average Target Mean Square: 0.478289


=== SUPER RESOLUTION TEST RESULTS ===
Test Loss: 17.112995
Test SNR: 2.7485 dB
Test MAE: 0.358543
Test MSE: 0.246857
Test NMSE: 0.516215
Test Pearson Correlation: 0.7732

Using callback metrics: {'test_loss': 17.112995147705078, 'test_sn

[{'test_loss': 17.112995147705078,
  'test_snr': 2.7484703063964844,
  'test_mae': 0.35854288935661316,
  'test_mse': 0.2468574345111847,
  'test_nmse': 0.5162146687507629,
  'test_pearson': 0.7732160687446594}]

In [6]:
if logger and isinstance(logger, pl.loggers.WandbLogger):
    logger.finalize('success')
    wandb.finish()

epoch,▁
test_loss,▁
test_mae,▁
test_mse,▁
test_nmse,▁
test_pearson,▁
test_snr,▁
trainer/global_step,▁▁▁▁▁▁▁
epoch,0
test_loss,17.113
test_mae,0.35854
